In [22]:
import json
import numpy as np
import os
import pandas as pd
import pymorphy2
import re
import sys
import shutil
import tensorflow as tf


from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from math import log
from sklearn.feature_extraction.text import CountVectorizer
from time import time

trained_size = 2000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from tensorflow.python.framework import ops


In [3]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def enum_sort(arr): 
    return [x[0] for x in sorted(enumerate(arr), key=lambda x:x[1], reverse=True)]

In [16]:
questions = pd.read_csv('quora_question_pairs_rus.csv', index_col=0).dropna()
questions.head()

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [18]:
train_texts = questions[:trained_size]['question2'].dropna().tolist()

In [65]:
morph=pymorphy2.MorphAnalyzer()

def preprocess_text(text): 
    low = text.lower()
    stripped = re.sub('!|\.|,|#|$|%|\\|\'|\(|\)|-|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~', '', low)
    words = stripped.split()
    return words

In [66]:
def lemmatize(text):
    return [morph.parse(word)[0].normal_form for word in preprocess_text(text)]

In [4]:
model_file = '/'.join(['w2v''model.model'])
model = KeyedVectors.load(model_file)

In [55]:
model['партнер'][10:20]

array([ 0.19693859, -0.2209126 , -0.04660673, -0.11634202,  0.15335335,
        0.1662537 ,  0.27540448, -0.00328762,  0.03214896,  0.20851499],
      dtype=float32)

In [62]:
def doc_to_vec_fasttext(doc_text):
    lemmas = lemmatize(doc_text)

    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    doc_vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model.vocab: 
            try:
                lemmas_vectors[idx] = model[lemma]
            except AttributeError as e: 
                print(e)
    if lemmas_vectors.shape[0] is not 0:  
        doc_vec = np.mean(lemmas_vectors, axis=0)
    return doc_vec

In [67]:
doc_to_vec_fasttext('По словам самой Греты, впервые она услышала об изменении климата в восемь лет')[10:20]

array([-0.02911203,  0.04704195, -0.07956814, -0.26593375,  0.01650164,
        0.12860344, -0.10586773,  0.02878121, -0.05393306,  0.01783438])

In [68]:
print('fasttext')
start = time()
fasttext_doc2vec_corpus = []
for doc in train_texts:
    fasttext_doc2vec_corpus.append(doc_to_vec_fasttext(doc))
time_fasttext = time() - start
print(f'TIME: {str(time_fasttext)}')
print(f' example of text: {train_texts[0]},\n first 10 values from its vector:\n {fasttext_doc2vec_corpus[0][:10]}')

fasttext
TIME: 4.4333319664001465
 example of text: что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад,
 first 10 values from its vector:
 [ 0.08526072 -0.13816322  0.22055048  0.07391119 -0.05820097  0.03983675
  0.1432986  -0.19306757 -0.33235623 -0.0051039  -0.23270215  0.050502
  0.09111848 -0.30432274  0.03272631  0.09067884 -0.0325922   0.03320344
 -0.12568103  0.0497015 ]
